In [7]:
# ==== PATCH: robust 'dt' creation (put right after reading CSVs) ====
import pandas as pd
import numpy as np

def add_dt_robust(df):
    out = df.copy()

    # 0) 이미 dt가 있으면 파싱만 보증
    if "dt" in out.columns:
        out["dt"] = pd.to_datetime(out["dt"], errors="coerce")

    # 1) 단일 일시 컬럼 후보
    if "dt" not in out.columns or out["dt"].isna().all():
        for col in ["일시", "datetime", "DateTime", "DATETIME", "timestamp", "Timestamp"]:
            if col in out.columns:
                out["dt"] = pd.to_datetime(out[col], errors="coerce")
                break

    # 2) '날짜' + '시간' 조합 (각종 포맷 견딤)
    if "dt" not in out.columns or out["dt"].isna().all():
        dcol = next((c for c in ["날짜","date","DATE","Date"] if c in out.columns), None)
        tcol = next((c for c in ["시간","hour","Hour","HOUR","time","TIME"] if c in out.columns), None)
        if dcol and tcol:
            d = pd.to_datetime(out[dcol], errors="coerce").dt.date

            # 시간: 숫자/문자/“13:00”/“13시” 모두 처리
            h_raw = out[tcol].astype(str).str.replace("시","", regex=False)
            # "13:00" 같은 건 시간만 추출
            h_try_time = pd.to_datetime(h_raw, errors="coerce").dt.hour
            h_try_num  = pd.to_numeric(h_raw, errors="coerce")
            h = h_try_num.fillna(h_try_time).fillna(0).astype(int).clip(0, 23)

            out["dt"] = pd.to_datetime(d) + pd.to_timedelta(h, unit="h")

    # 3) 최종 확인
    if "dt" not in out.columns or out["dt"].isna().all():
        raise RuntimeError(f"dt 생성 실패. 사용 가능한 컬럼: {list(df.columns)}")

    return out

# CSV 읽은 직후에 호출하세요
train_df = add_dt_robust(train_df)
test_df  = add_dt_robust(test_df)
print("dt ok | train:", train_df["dt"].min(), "→", train_df["dt"].max(),
      "| test:", test_df["dt"].min(), "→", test_df["dt"].max())


dt ok | train: 2024-06-01 00:00:00 → 2024-08-24 23:00:00 | test: 2024-08-25 00:00:00 → 2024-08-31 23:00:00


In [22]:
def ensure_dt(df):
    """dt 만들기: dt>일시>날짜+시간>기타 후보 순으로 탐색."""
    out = df.copy()

    # 0) dt가 이미 있으면 검증 후 사용
    if "dt" in out.columns:
        tmp = pd.to_datetime(out["dt"], errors="coerce")
        if tmp.notna().any():
            out["dt"] = tmp
            return out

    # 1) '일시' 단일 컬럼 우선
    if "일시" in out.columns:
        out["dt"] = pd.to_datetime(out["일시"], errors="coerce")
        if out["dt"].notna().any():
            return out

    # 2) '날짜' + '시간' 조합
    if {"날짜","시간"}.issubset(out.columns):
        d = pd.to_datetime(out["날짜"], errors="coerce")

        h_raw = out["시간"].astype(str).str.replace("시","", regex=False)
        # "0~23" 형태면 숫자, 그 외는 날짜형 파싱 → hour 추출
        h = (pd.to_numeric(h_raw, errors="coerce")
             .fillna(pd.to_datetime(h_raw, errors="coerce").dt.hour)
             .fillna(0).astype(int).clip(0, 23))

        out["dt"] = d + pd.to_timedelta(h, unit="h")
        return out

    # 3) 기타 후보 이름 시도
    for cand in ["datetime","timestamp","일자","기준일시","date","Date","DATE"]:
        if cand in out.columns:
            out["dt"] = pd.to_datetime(out[cand], errors="coerce")
            if out["dt"].notna().any():
                return out

    # 4) 객체형 중 70% 이상 파싱되면 채택
    best = None; best_ratio = 0.0
    for c in out.columns:
        if out[c].dtype == object:
            parsed = pd.to_datetime(out[c], errors="coerce")
            r = parsed.notna().mean()
            if r > best_ratio:
                best, best_ratio = parsed, r
    if best is not None and best_ratio >= 0.7:
        out["dt"] = best
        return out

    # 디버깅 도움 출력
    raise KeyError(f"[ensure_dt] 시간 컬럼을 찾지 못했어요. 현재 컬럼들: {list(out.columns)}")

def ensure_submit_ids(df):
    """제출용 ID(날짜/시간)가 없으면 dt에서 복원."""
    out = df.copy()
    if "dt" in out.columns:
        if "날짜" not in out.columns:
            out["날짜"] = out["dt"].dt.strftime("%Y-%m-%d")
        if "시간" not in out.columns:
            out["시간"] = out["dt"].dt.hour
    return out



In [23]:
train_df = ensure_dt(train_df)
test_df  = ensure_dt(test_df)

# 2) 캘린더/파생
train_df = add_simple_extra_feats(add_calendar_feats(train_df))
test_df  = add_simple_extra_feats(add_calendar_feats(test_df))

# 3) 제출용 ID 보장(날짜/시간 생성)
test_df  = ensure_submit_ids(test_df)

In [24]:
for col_to_drop in ["dt","일시","datetime","timestamp","일자","기준일시","Date","DATE","date","날짜","시간"]:
    if col_to_drop in FEATS:
        FEATS.remove(col_to_drop)

In [26]:
print("train_df cols:", list(train_df.columns))
print("test_df  cols:", list(test_df.columns))

train_df cols: ['num_date_time', '건물번호', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)']
test_df  cols: ['num_date_time', '건물번호', '일시', '기온(°C)', '강수량(mm)', '풍속(m/s)', '습도(%)']


In [32]:
# ============================================
# CatBoost (categorical-aware) E2E 튜닝 & 제출 (num_date_time/일시 지원)
# - 데이터 적재 → dt/날짜/시간 복원 → 피처엔지니어링
# - 시간 기반 홀드아웃(마지막 20%) + RandomizedSearchCV
# - 베스트 파라미터로 전체 재학습 → test 예측 → 제출 저장
# ============================================
import os, warnings, joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import RandomizedSearchCV, PredefinedSplit
from sklearn.metrics import make_scorer
from scipy.stats import randint, uniform
from catboost import CatBoostRegressor

warnings.filterwarnings("ignore")

# -----------------------------
# 0) 경로/설정
# -----------------------------
TRAIN_CSV = r"C:\Users\user\Downloads\open (1)\train.csv"
TEST_CSV  = r"C:\Users\user\Downloads\open (1)\test.csv"

ID_COLS   = ["건물번호", "날짜", "시간"]   # 제출 키
TARGET    = "전력소비량(kWh)"
SUBMIT_TARGET_NAME = "전력예측량(kWh)"     # 대회에서 요구하는 컬럼명
CDD_BASE_TEMP = 26

N_ITER_RS  = 25      # RandomizedSearch 반복 수
EARLY_STOP = 100     # CatBoost early stopping 라운드
VAL_FRAC   = 0.2     # 마지막 20% 구간을 검증으로 사용

# -----------------------------
# 1) 유틸: dt/날짜/시간 복원
# -----------------------------
def ensure_dt_from_any(df: pd.DataFrame) -> pd.DataFrame:
    """일시 / num_date_time 중 아무거나로 dt를 만들고 날짜/시간까지 복원."""
    out = df.copy()

    # 이미 dt가 정상적이면 유지
    if "dt" in out.columns and pd.to_datetime(out["dt"], errors="coerce").notna().any():
        pass
    elif "일시" in out.columns:
        out["dt"] = pd.to_datetime(out["일시"], errors="coerce")
    elif "num_date_time" in out.columns:
        s = out["num_date_time"]
        s_num = pd.to_numeric(s, errors="coerce")

        if s_num.notna().all() and s_num.min() > 1e12:
            # epoch milliseconds
            out["dt"] = pd.to_datetime(s_num, unit="ms", origin="unix")
        elif s_num.notna().all() and s_num.min() > 1e9:
            # epoch seconds
            out["dt"] = pd.to_datetime(s_num, unit="s", origin="unix")
        else:
            # yyyymmddHH or yyyymmddHHMM 형태 시도
            s_str = s.astype(str).str.replace(r"\D", "", regex=True)
            dt = pd.to_datetime(s_str, format="%Y%m%d%H", errors="coerce")
            mask_na = dt.isna()
            if mask_na.any():
                dt2 = pd.to_datetime(s_str[mask_na], format="%Y%m%d%H%M", errors="coerce")
                dt[mask_na] = dt2
            out["dt"] = dt
    else:
        raise ValueError("dt를 만들 기준 열(일시/num_date_time)이 없습니다.")

    # 제출 키 생성
    out["날짜"] = out["dt"].dt.strftime("%Y-%m-%d")
    out["시간"] = out["dt"].dt.hour.astype(int)
    return out

# -----------------------------
# 2) 피처엔지니어링
# -----------------------------
def add_calendar_feats(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    out = ensure_dt_from_any(out)
    out["hour"]      = out["dt"].dt.hour
    out["month"]     = out["dt"].dt.month
    out["dayofyear"] = out["dt"].dt.dayofyear
    out["weekday"]   = out["dt"].dt.weekday
    out["is_weekend"]= (out["weekday"] >= 5).astype(int)
    out["hour_sin"]  = np.sin(2*np.pi*out["hour"]/24)
    out["hour_cos"]  = np.cos(2*np.pi*out["hour"]/24)
    if "is_holiday" not in out.columns:
        out["is_holiday"] = 0
    return out

def add_simple_extra_feats(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    # CDD류
    if "기온(°C)" in out.columns:
        temp = pd.to_numeric(out["기온(°C)"], errors="coerce")
        cdd = np.clip(temp - CDD_BASE_TEMP, 0, None)
        out["CDD"] = cdd
        if "일사(MJ/m2)" in out.columns:
            rad = pd.to_numeric(out["일사(MJ/m2)"], errors="coerce")
            out["CDD_x_rad"] = cdd * rad
        if "습도(%)" in out.columns:
            hum = pd.to_numeric(out["습도(%)"], errors="coerce")
            out["CDD_humid_adj"] = cdd * (1 + hum/100.0)
    # 용량 파생 (없으면 자동 스킵)
    for col in ["태양광용량(kW)","ESS저장용량(kWh)","PCS용량(kW)"]:
        if col in out.columns:
            v = pd.to_numeric(out[col], errors="coerce").fillna(0)
            out[f"has_{'pv' if '태양' in col else ('ess' if 'ESS' in col else 'pcs')}"] = (v > 0).astype(int)
            out[f"log1p_{col}"] = np.log1p(v)
    return out

# -----------------------------
# 3) 데이터 적재 & 전처리
# -----------------------------
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

train_df = add_simple_extra_feats(add_calendar_feats(train_df))
test_df  = add_simple_extra_feats(add_calendar_feats(test_df))

# 1) lag/rolling 생성 (누수 없이)
def add_lag_roll_feats(train, test, key_cols=("건물번호","dt"), y_col="전력소비량(kWh)"):
    assert y_col in train.columns, "train에 타깃이 필요합니다."
    use_cols = list(key_cols) + [y_col]
    aux = pd.concat([train[use_cols], test[list(key_cols)]], ignore_index=True)
    aux = aux.sort_values(list(key_cols)).reset_index(drop=True)

    g = aux.groupby(key_cols[0], group_keys=False)
    aux["cons_lag1"]     = g[y_col].shift(1)
    aux["cons_lag_24h"]  = g[y_col].shift(24)
    aux["cons_lag_168h"] = g[y_col].shift(168)
    aux["cons_mean_24h"] = g[y_col].shift(1).rolling(24).mean()

    # 테스트 부분만 추출 후 원본 test와 병합
    test_key = test[list(key_cols)].copy()
    test_aug = test_key.merge(aux.drop(columns=[y_col]), on=list(key_cols), how="left")
    keep = [c for c in test.columns if c not in key_cols]
    test_aug = test_aug.merge(test[keep], left_index=True, right_index=True, how="left")
    return test_aug

# 2) 사용법 (네 스크립트의 피처엔지니어링 직후에 추가)
# train_df, test_df에 dt까지 생성된 상태여야 함
test_df = add_lag_roll_feats(train_df, test_df, key_cols=("건물번호","dt"), y_col=TARGET)

# 3) FEATS에 lag/rolling 이름을 포함
extra_lags = ["cons_lag1","cons_lag_24h","cons_lag_168h","cons_mean_24h"]
FEATS = sorted(set(FEATS + [c for c in extra_lags if c in test_df.columns]))
print("[INFO] #features with lags:", len(FEATS))

# 사용 예시: add_calendar_feats/add_simple_extra_feats 이후
train_df = add_calendar_feats(train_df)
test_df  = add_calendar_feats(test_df)
train_df = add_simple_extra_feats(train_df)
test_df  = add_simple_extra_feats(test_df)
test_df  = add_lag_roll_feats(train_df, test_df, key_cols=("건물번호","dt"), y_col=TARGET)

# 그런 다음 FEATS에 ['cons_lag1','cons_lag_24h','cons_lag_168h','cons_mean_24h']를 포함시키면 됨.

# -----------------------------
# 4) 피처 집합 고정 (train 기준) + test 누락 컬럼 보충
# -----------------------------
drop_cols = set(ID_COLS + [TARGET, "dt", "일시", "num_date_time"])
FEATS = [c for c in train_df.columns if c not in drop_cols]

# test에 없는 피처는 NaN으로 생성(학습/추론 동일 스키마 보장)
missing_in_test = [c for c in FEATS if c not in test_df.columns]
for c in missing_in_test:
    test_df[c] = np.nan

# 최종 정렬
FEATS = list(FEATS)

# CatBoost 범주형: 문자열/카테고리 열만 자동 지정 (이번 스키마에선 거의 없음)
CAT_COLS = [c for c in FEATS if str(train_df[c].dtype) in ("object","category")]
for c in CAT_COLS:
    train_df[c] = train_df[c].astype("category")
    test_df[c]  = test_df[c].astype("category")

print(f"[INFO] #train={len(train_df)}, #test={len(test_df)}")
print(f"[INFO] #features={len(FEATS)}, cats={CAT_COLS}")

# -----------------------------
# 5) 시간 기반 검증 분할 (마지막 20%를 검증)
# -----------------------------
assert "dt" in train_df.columns, "dt 생성 실패"
dt_cut = train_df["dt"].quantile(1.0 - VAL_FRAC)
train_mask = train_df["dt"] < dt_cut
valid_mask = ~train_mask

X = train_df[FEATS].copy()
y = train_df[TARGET].copy()

test_fold = np.full(len(train_df), -1)
test_fold[valid_mask.values] = 0
ps = PredefinedSplit(test_fold=test_fold)

X_tr, y_tr = X.iloc[train_mask.values], y.iloc[train_mask.values]
X_va, y_va = X.iloc[valid_mask.values], y.iloc[valid_mask.values]

# -----------------------------
# 6) 하이퍼파라미터 탐색(RS) + early stopping
# -----------------------------
def rmse_np(y_true, y_pred):
    y_true = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))

def neg_rmse(y_true, y_pred):
    return -rmse_np(y_true, y_pred)

from sklearn.metrics import make_scorer
scorer = make_scorer(neg_rmse, greater_is_better=True)

base = CatBoostRegressor(
    loss_function="RMSE",
    random_seed=42,
    verbose=False
)

param_dist = {
    "depth": randint(5, 10),                       # 5~9
    "learning_rate": uniform(0.03, 0.09),          # 0.03~0.12
    "l2_leaf_reg": uniform(1.0, 9.0),              # 1~10
    "bagging_temperature": uniform(0.0, 1.0),      # 0~1
    "random_strength": uniform(0.0, 1.0),          # 0~1
    "rsm": uniform(0.6, 0.4),                      # 0.6~1.0
    "min_data_in_leaf": randint(1, 64),            # 1~63
    "iterations": randint(700, 1601),              # 700~1600
}

rs = RandomizedSearchCV(
    estimator=base,
    param_distributions=param_dist,
    n_iter=N_ITER_RS,
    cv=ps,
    scoring=scorer,
    n_jobs=-1,
    refit=True,
    verbose=1,
    random_state=42,
)

X_valid, y_valid = X.iloc[valid_mask.values], y.iloc[valid_mask.values]
fit_params = {
    "cat_features": CAT_COLS if len(CAT_COLS) > 0 else None,
    "eval_set": (X_valid, y_valid),
    "early_stopping_rounds": EARLY_STOP,
    "use_best_model": True
}
print(">>> RandomizedSearchCV 시작")
rs.fit(X, y, **fit_params)

print("\n[검색 완료]")
print("best_score_ (−RMSE):", rs.best_score_)
print("best_params_:", rs.best_params_)

cvres = pd.DataFrame(rs.cv_results_)
cvres.to_csv("cat_rs_results.csv", index=False, encoding="utf-8-sig")
print("saved -> cat_rs_results.csv")

# -----------------------------
# 7) 베스트 파라미터로 전데이터 재학습
# -----------------------------
best_model_cv = rs.best_estimator_
try:
    best_iter = best_model_cv.get_best_iteration()
except Exception:
    best_iter = rs.best_params_.get("iterations", 1000)

final_params = rs.best_params_.copy()
final_params.update(dict(
    iterations=max(50, int(best_iter)),
    loss_function="RMSE",
    random_seed=42,
    verbose=False
))

final_model = CatBoostRegressor(**final_params)
final_model.fit(
    train_df[FEATS], train_df[TARGET],
    cat_features=CAT_COLS if len(CAT_COLS) > 0 else None
)

art = {
    "model": final_model,
    "feature_order": FEATS,
    "cat_cols": CAT_COLS,
    "best_params_cv": rs.best_params_,
    "best_cv_neg_rmse": rs.best_score_,
    "best_cv_iter": best_iter
}
joblib.dump(art, "catboost_tuned.joblib")
print("saved -> catboost_tuned.joblib")

# -----------------------------
# 8) 테스트 예측 & 제출 저장
# -----------------------------
X_test = test_df[FEATS].copy()
for c in CAT_COLS:
    X_test[c] = X_test[c].astype("category")

pred = final_model.predict(X_test)

submit = test_df[ID_COLS].copy()
submit[SUBMIT_TARGET_NAME] = pred
submit.to_csv("submission_cat_tuned.csv", index=False, encoding="utf-8-sig")
print("saved -> submission_cat_tuned.csv")
print(submit.head())


[INFO] #features with lags: 21
[INFO] #train=204000, #test=16800
[INFO] #features=17, cats=[]
>>> RandomizedSearchCV 시작
Fitting 1 folds for each of 25 candidates, totalling 25 fits

[검색 완료]
best_score_ (−RMSE): -3775.9214454021044
best_params_: {'bagging_temperature': np.float64(0.22210781047073025), 'depth': 7, 'iterations': 933, 'l2_leaf_reg': np.float64(4.038536542632651), 'learning_rate': np.float64(0.11486187335212672), 'min_data_in_leaf': 54, 'random_strength': np.float64(0.21876421957307024), 'rsm': np.float64(0.8232408008069365)}
saved -> cat_rs_results.csv
saved -> catboost_tuned.joblib
saved -> submission_cat_tuned.csv
   건물번호          날짜  시간   전력예측량(kWh)
0     1  2024-08-25   0  3089.855362
1     1  2024-08-25   1  3425.569081
2     1  2024-08-25   2  3172.811520
3     1  2024-08-25   3  3617.365304
4     1  2024-08-25   4  3280.049380
